In [1]:
%matplotlib inline
import pandas as pd
import os
import numpy as np

In [2]:
pdX = pd.read_pickle(os.path.join('data', 'our_df2.pickled'))
y = pd.read_pickle(os.path.join('data', 'ytrain.pkl'))

In [3]:
print(pdX.shape)
print(len(y)) # y is 1-dim

(17253, 337)
17254


In [4]:
import xgboost as xg

# suggested params
param = {
    'bst:eta' : [0.002, 0.02],
    'bst:subsample' : [0.5,1],
    'bst:mim_child_weight' : [0.1,10],
    'bst:max_depth' : [2,30],
    'bst:gamma' : [0,1],
    'objective':'binary:logistic'
}

# create booster

tb = xg.XGBClassifier(learning_rate=0.002)

ImportError: No module named xgboost

In [30]:
# preprocess data here

# map(clean)

In [33]:
X = xg.DMatrix(np.array(pdX))
# compute error - assuming we have Xv i yv
# we use tb.fit(X, y, eval_set=[(Xv,yv)], eval_metric=’logloss’, verbose=True)
# and then tb.evals_result() to get valitation metrics
tb.fit(X, y)


Exception AttributeError: "'DMatrix' object has no attribute 'handle'" in <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x7f7ff3f4d6d0>> ignored


TypeError: float() argument must be a string or a number